# Overview
This notebook will examine quantitative 7T MRI measures projected onto hippocampal midthickness surfaces and averaged across 10 subjects. At the end, we will combine this with Histology data and examine conserved features

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import sys
sys.path.insert(1, '/export03/data/opt/hippunfold_toolbox/hippunfold_toolbox')
import plotting
import utils
import copy
import glob

In [2]:
micapipe_dir = '../sourcedata/Supersession_PNI'
hippunfold_dir = '../hippunfold/PNI_v1.3.0_super/hippunfold'

subs = ['PNC002', 'PNC003', 'PNC006', 'PNC007', 'PNC009', 'PNC010', 'PNC015', 'PNC016', 'PNC018', 'PNC019']
ses = ''
hemis = ['L','R']
labels = ['hipp','dentate'] 

# here we will generate multiple depth-wise surfaces
layers = np.linspace(-0.25,1.25,num=25)
gm = np.where(np.logical_and(layers>=0,  layers <=1))[0]
ind = [range(7262), range(7262,7262+1788)]

In [ ]:
features = ['T1map','MTR','T2star','FA','ADC']
for s,sub in enumerate(subs):
    cmd = f'mkdir -p {hippunfold_dir}/sub-{sub}/surf/depths'
    !{cmd}
    for h,hemi in enumerate(hemis):
        for l,layer in enumerate(layers):
            cmd1 = f'wb_command -surface-cortex-layer '\
                f'{hippunfold_dir}/sub-{sub}/surf/sub-{sub}_hemi-{hemi}_space-corobl_den-0p5mm_label-hipp_inner.surf.gii '\
                f'{hippunfold_dir}/sub-{sub}/surf/sub-{sub}_hemi-{hemi}_space-corobl_den-0p5mm_label-hipp_outer.surf.gii '\
                f'{layer} '\
                f'{hippunfold_dir}/sub-{sub}/surf/depths/sub-{sub}_hemi-{hemis[s]}_layer-{layer}.surf.gii'
            !{cmd1}
            for f,feature in enumerate(features):
                cmd2 = f'wb_command -volume-to-surface-mapping '\
                    f'{micapipe_dir}/sub-{sub}/anat/sub-{sub}_space-nativepro_{feature}.nii.gz '\
                    f'{hippunfold_dir}/sub-{sub}/surf/depths/sub-{sub}_hemi-{hemi}_layer-{layer}.surf.gii '\
                    f'{hippunfold_dir}/sub-{sub}/surf/depths/sub-{sub}_hemi-{hemi}_layer-{layer}_{feature}.shape.gii '\
                    f'-trilinear'
                !{cmd2}

In [ ]:
features = ['T1map','MTR','T2star','FA','ADC']
hipp_dat = np.zeros([7262,2,len(subs),len(features),len(layers)])*np.nan

for f,feature in enumerate(features):
    for s,sub in enumerate(subs):
        for h,hemi in enumerate(hemis):
            for l,layer in enumerate(layers):
                try:
                    d = nib.load(f'{hippunfold_dir}/sub-{sub}/surf/depths/sub-{sub}_hemi-{hemi}_layer-{layer}_{feature}.shape.gii')
                    hipp_dat[:,h,s,f,l] = d.darrays[0].data
                except:
                    print(f'sub-{sub}_{hemi}_{feature} not found')

In [ ]:
cdata = np.nanmean(hipp_dat,axis=2)
plotting.surfplot_canonical_foldunfold(cdata[:,:,:], color_bar=('right'), labels=['hipp'], share='row', tighten_cwindow=True, embed_nb=True)

In [ ]:
hipp_dat.shape

In [ ]:
cdata = np.nanmean(hipp_dat,axis=(1,2))
plotting.surfplot_canonical_foldunfold(np.nanmean(cdata[:,:,:,gm],axis=3), color_bar=('right'), hemis=['L'], labels=['hipp'], share='row', tighten_cwindow=True, embed_nb=True)

In [ ]:
for i in range(8):
    plotting.surfplot_canonical_foldunfold(np.nanmean(cdata[:,:,:,i,gm],axis=3), color_bar=('right'), hemis=['L'], size=[350,270], share='row', tighten_cwindow=True, embed_nb=True, screenshot=True, filename=f'{i}.png')

## Add histology data
Since the histology has only one hemisphere and no DG, we will heep only this data from MRI

In [ ]:
sys.path.insert(1, '/data/mica1/01_programs/micapipe-v0.2.0/functions')
from build_mpc import build_mpc
from brainspace.gradient import GradientMaps

In [ ]:
downsampled_histo = np.load("../checkpoints/struct-HISTO-proc-midsurfaces.npy")
struct_data = np.concatenate((downsampled_histo,np.nanmean(cdata[:,:,:],axis=1)),axis=1)

In [ ]:
# correlation between features
feat_corr = np.corrcoef(struct_data.T)
allfeatures = ['Merker', 'PLI-transmittance', 'Blockface', 'Bieloschowsky', 'Calbindin', 'Calretinin', 'Parvalbumin', 'Thionin', 'PD',\
               'qR1', 'qR2star', 'thickness', 'gyrification', 'curvature', 'T1map', 'MTR', 'T2star', 'FA', 'ADC']

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(feat_corr, vmin=-1, vmax=1, cmap='bwr')
plt.yticks(ticks=range(len(allfeatures)),labels=allfeatures);
plt.xticks(ticks=range(len(allfeatures)),labels=allfeatures, rotation=90);

In [ ]:
# gradient decomposition
ngrads=5
mmgm = GradientMaps(n_components=ngrads, kernel='cosine', random_state=0)
mmgm.fit(struct_data[:,:], sparsity=0.1)
plotting.surfplot_canonical_foldunfold(mmgm.gradients_, labels=['hipp'], hemis=['L'], size=[350,270], color_bar='right', share='row', tighten_cwindow=True, embed_nb=True)

In [ ]:
plt.plot(np.arange(ngrads)+1,mmgm.lambdas_)

In [ ]:
mmgm.gradients_.shape